In [2]:
import pandas as pd
import requests
import re
import numpy as np

In [84]:
# Souce of Transportation Data (City OpenData)
# DC bus - https://opendata.dc.gov/datasets/metro-stations-regional/data
# DC subway - https://opendata.dc.gov/datasets/54018b7f06b943f2af278bbe415df1de_52/data

# Bos bus https://geo-massdot.opendata.arcgis.com/datasets/2c00111621954fa08ff44283364bba70_0
# Bos subway (MBTA Rapid Transit Stops) http://maps-massgis.opendata.arcgis.com/datasets/a9e4d01cbfae407fbf5afe67c5382fde_0

# Ny bus https://data.cityofnewyork.us/Transportation/Bus-Stop-Shelters/qafz-7myz 
# NY subway https://data.cityofnewyork.us/Transportation/Subway-Stations/arq3-7z49

In [4]:
ny_subway = pd.read_csv('NY_SUBWAY_STATION_01_13SEPT2010.csv', encoding='latin1')
ny_bus = pd.read_csv('NY_Bus_Stop_Shelter.csv', encoding='latin1')
bos_subway = pd.read_csv('BT_MBTA_Rapid_Transit_Stops.csv', encoding='latin1')
bos_bus = pd.read_csv('NY_Bus_Stop_Shelter.csv', encoding='latin1')
dc_subway = pd.read_csv('DC_Metro_Stations_Regional.csv', encoding='latin1')
dc_bus = pd.read_csv('DC_Metro_Bus_Stops.csv', encoding='latin1')

In [34]:
ny_subway_1 = ny_subway[['NAME','X','Y','LINE']]
ny_subway_1['city'] = 'ny'
bos_subway_1 = bos_subway[['STATION','X','Y','LINE']]
bos_subway_1['city'] = 'bos'
dc_subway_1 = dc_subway[['NAME','X','Y','LINE']]
dc_subway_1['city'] = 'dc'

In [63]:
ny_subway_2 = ny_subway_1.rename(columns={'NAME':'station_name','X':'station_lon','Y':'station_lat','LINE':'line'})
bos_subway_2 = bos_subway_1.rename(columns={'STATION':'station_name','X':'station_lon','Y':'station_lat','LINE':'line'})
dc_subway_2 = dc_subway_1.rename(columns={'NAME':'station_name','X':'station_lon','Y':'station_lat','LINE':'line'})

In [70]:
subway = pd.concat([ny_subway_2, bos_subway_2, dc_subway_2], ignore_index=True)
subway['type'] = 'subway'

In [78]:
ny_bus_1 = ny_bus[['SHELTER_ID','LONGITUDE','LATITUDE']]
ny_bus_1['line'] = np.NaN
bos_bus_1 = bos_bus[['SHELTER_ID','LONGITUDE','LATITUDE']]
bos_bus_1['line'] = np.NaN
dc_bus_1 = dc_bus[['BSTP_GEO_ID','X','Y']]
dc_bus_1['line'] = np.NaN

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [80]:
ny_bus_1['city'] = 'ny'
bos_bus_1['city'] = 'bos'
dc_bus_1['city'] = 'dc'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [81]:
ny_bus_2 = ny_bus_1.rename(columns={'SHELTER_ID':'stop_name','LONGITUDE':'stop_lon','LATITUDE':'stop_lat'})
bos_bus_2 = bos_bus_1.rename(columns={'SHELTER_ID':'stop_name','LONGITUDE':'stop_lon','LATITUDE':'stop_lat'})
dc_bus_2 = dc_bus_1.rename(columns={'BSTP_GEO_ID':'stop_name','X':'stop_lon','Y':'stop_lat'})

In [82]:
bus = pd.concat([ny_bus_2, bos_bus_2, dc_bus_2], ignore_index=True)
bus['type'] = 'bus'

In [56]:
def get_subway_zipcode(x):
    lat = x['station_lat']
    lon = x['station_lon']
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        response_data = response.json()
        for i in range(len(response_data['results'])):
            address = response_data['results'][i]['formatted_address']
            match = re.search(r'\d{5}',address)
            if match:
                return int(match.group())
        return np.NaN
    else:
        return None

subway['zipcode'] = subway.apply(get_subway_zipcode, axis=1)
subway['zipcode'] = subway.apply(lambda x: str(x['zipcode']).zfill(5),axis=1)
subway.to_csv('subway_with_zipcode.csv')

In [67]:
def get_bus_zipcode(x):
    lat = x['stop_lat']
    lon = x['stop_lon']
    url = f'https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        response_data = response.json()
        for i in range(len(response_data['results'])):
            address = response_data['results'][i]['formatted_address']
            match = re.search(r'\d{5}',address)
            if match:
                return int(match.group())
        return np.NaN
    else:
        return None
    
bus['zipcode'] = bus.apply(get_bus_zipcode, axis=1)
bus['zipcode'] = bus.apply(lambda x: str(x['zipcode']).zfill(5),axis=1)
bus.to_csv('bus_with_zipcode.csv')

In [93]:
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21822 entries, 0 to 21821
Data columns (total 8 columns):
Unnamed: 0    21822 non-null int64
stop_name     21818 non-null object
stop_lon      21822 non-null float64
stop_lat      21822 non-null float64
line          0 non-null float64
city          21822 non-null object
type          21822 non-null object
zipcode       21822 non-null object
dtypes: float64(3), int64(1), object(4)
memory usage: 1.3+ MB


In [94]:
subway.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 8 columns):
Unnamed: 0      728 non-null int64
station_name    728 non-null object
station_lon     728 non-null float64
station_lat     728 non-null float64
line            728 non-null object
city            728 non-null object
type            728 non-null object
zipcode         728 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 45.6+ KB


In [7]:
# with open('ny_new_york_zip_codes_geo.min.json') as fp:
#     zip_ny = fp.read()
# with open('ma_massachusetts_zip_codes_geo.min.json') as fp:
#     zip_ma = fp.read()
# with open('dc_district_of_columbia_zip_codes_geo.min.json') as fp:
#     zip_dc = fp.read()
# import json
# zip_ma_json = json.loads(zip_ma)
# zip_dc_json = json.loads(zip_dc)

In [1]:
# from shapely.geometry import Point
# from shapely.geometry.polygon import Polygon

# point = Point(0.5, 0.5)
# polygon = Polygon([(0, 0), (0, 1), (1, 1), (1, 0)])
# print(polygon.contains(point))
# p2 = Point(polygon.centroid)

True


In [79]:
# lis = zip_ma_json['features'][0]['geometry']['coordinates'][0]
# polygon = Polygon(lis)
# point = Point([-70.610388, 41.633897])
# polygon.contains(point)

False

In [52]:
# def find_zipcode(x):
#     lat = x['station_lat']
#     lng = x['station_lon']
#     url = prefix + f'?lat={lat}&lng={lng}'
#     soup = BeautifulSoup(requests.get(url).content, 'lxml')
#     tr = soup.find('table',id='tableInfo50').find_all('tr')
#     zipcode = tr[4].find_all('td')[1].text.strip()
#     print(zipcode)
#     return zipcode